In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv('./user_item_vec.csv')

In [4]:
df.head()

,user,"livestock|pp|goat,sheep",crop|fertilizer_management|okra,"livestock|dm|cattle,buffalo",crop|fertilizer_management|paddy,"livestock|dm|chicken,duck",crop|nursery_management|paddy,"livestock|bm|goat,sheep","livestock|dm|cattle,buffalo,goat,sheep","crop|fertilizer_management|pulses,rabi",...,crop|nan|nan,"crop|harvest_management|blackgram,greengram",crop|harvest_management|groundnut,crop|harvest_management|cotton,crop|nursery_management|groundnut,crop|fertilizer_management|potato,"crop|nursery_management|greengram,blackgram",crop|nursery_management|pulses,crop|nursery_management|hybrids,crop|fertilizer_management|pulses
0,7894892401,-0.025565,0.182844,-0.0179,-0.21993,-0.156819,-0.150627,0.838736,-0.097153,0.702081,...,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.288668
1,9938189160,-0.025565,0.182844,-0.0179,-0.21993,-0.156819,-0.150627,0.838736,-0.097153,0.702081,...,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.288668
2,9938089536,-0.025565,0.182844,-0.0179,-0.21993,-0.156819,-0.150627,0.838736,-0.097153,0.702081,...,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.288668
3,9777234388,-0.025565,0.182844,-0.0179,-0.21993,-0.156819,-0.150627,0.838736,-0.097153,0.702081,...,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.288668
4,9776210885,-0.025565,0.182844,-0.0179,-0.21993,-0.156819,-0.150627,0.838736,-0.097153,0.702081,...,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.281029,-0.288668


In [7]:
feature_cols = list(df.columns)[1:]

In [8]:
X = df[feature_cols].values

In [13]:
np.unique(X)

array([-0.28866777, -0.28102925, -0.28033565, -0.28030097, -0.28022005,
       -0.27860165, -0.26700141, -0.21992959, -0.15681924, -0.1506273 ,
       -0.14064759, -0.09715335, -0.02556545, -0.01790005,  0.04277623,
        0.18284399,  0.70208138,  0.83873558])

In [10]:
def calc_sum_square_dist(X,centroids,labels):
    def calc_square_dist(x1,x2):
        return np.sum(np.square(x1-x2))
    fin_sum = 0
    for x,l in zip(X,labels):
        fin_sum += calc_square_dist(x,centroids[l])
        
    return fin_sum/len(X)

In [11]:
k_vs_loss = {}
for k in range(1,20):
    kmeans = KMeans(n_clusters=k).fit(X)
    label = kmeans.labels_
    centroid = kmeans.cluster_centers_
    err_loss = calc_sum_square_dist(X,centroid,label)
    k_vs_loss[k] = err_loss

/tmp/ipykernel_106744/46946783.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=k).fit(X)
/tmp/ipykernel_106744/46946783.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=k).fit(X)
/tmp/ipykernel_106744/46946783.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=k).fit(X)
/tmp/ipykernel_106744/46946783.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=k).fit(X)
/tmp/ipykernel_106744/46946783.py:3: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=

In [12]:
k_vs_loss

{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0}